In [1]:
from gurobipy import Model, GRB

In [2]:
# Initialize the model
m = Model("Cruise_Reservations")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [3]:
# First stage decision variables
x = m.addVars(3, vtype=GRB.INTEGER, name="x")  # Regular reservations for package i
y = m.addVars(3, vtype=GRB.INTEGER, name="y")  # VIP reservations for package i

In [4]:
# Anticipated demand for each scenario (i, n)
demand_regular = {(1, 1): 79, (1, 2): 83, (1, 3): 87, (1, 4): 91, (1, 5): 95,
                  (2, 1): 50, (2, 2): 53, (2, 3): 56, (2, 4): 59, (2, 5): 62,
                  (3, 1): 54, (3, 2): 64, (3, 3): 64, (3, 4): 74, (3, 5): 84}
demand_vip = {(1, 1): 40, (1, 2): 42, (1, 3): 44, (1, 4): 46, (1, 5): 50,
              (2, 1): 30, (2, 2): 33, (2, 3): 36, (2, 4): 39, (2, 5): 42,
              (3, 1): 29, (3, 2): 33, (3, 3): 37, (3, 4): 41, (3, 5): 45}

In [5]:
# Prices for each package
prices = {1: {'Regular': 4180, 'VIP': 5630},
          2: {'Regular': 2230, 'VIP': 3460},
          3: {'Regular': 2390, 'VIP': 3520}}

In [6]:
# Second stage decision variables for each scenario n
x_scenarios = m.addVars(3, 5, vtype=GRB.INTEGER, name="x_scenarios")  # Regular, scenario n
y_scenarios = m.addVars(3, 5, vtype=GRB.INTEGER, name="y_scenarios")  # VIP, scenario n

In [8]:
# Adjusted indices for the objective function
objective_terms = [(prices[i]['Regular'] * x_scenarios[i-1, n-1] +
                    prices[i]['VIP'] * y_scenarios[i-1, n-1]) for i in range(1, 4) for n in range(1, 6)]
m.setObjective(1/5 * sum(objective_terms), GRB.MAXIMIZE)

In [10]:

# Add constraints
# Capacity constraints
for i in range(3):  # Adjusted to 0-based index
    m.addConstr(x[i] + y[i] <= 175, f"Capacity_Package_{i+1}")

# Demand constraints
for i in range(3):  # Adjusted to 0-based index
    for n in range(5):  # Adjusted to 0-based index
        m.addConstr(x_scenarios[i, n] <= demand_regular[i+1, n+1], f"Demand_Regular_{i+1}_{n+1}")
        m.addConstr(y_scenarios[i, n] <= demand_vip[i+1, n+1], f"Demand_VIP_{i+1}_{n+1}")

# VIP percentage constraints
for i in range(3):  # Adjusted to 0-based index
    for n in range(5):  # Adjusted to 0-based index
        m.addConstr(y_scenarios[i, n] <= 0.4 * (x_scenarios[i, n] + y_scenarios[i, n]),
                    f"VIP_Percentage_{i+1}_{n+1}")

# Linking first stage decisions to second stage
for i in range(3):  # Adjusted to 0-based index
    for n in range(5):  # Adjusted to 0-based index
        m.addConstr(x_scenarios[i, n] <= x[i], f"Link_x_{i+1}_{n+1}")
        m.addConstr(y_scenarios[i, n] <= y[i], f"Link_y_{i+1}_{n+1}")

# Non-negativity constraints
for i in range(3):  # Adjusted to 0-based index
    m.addConstr(x[i] >= 0, f"NonNeg_x_{i+1}")
    m.addConstr(y[i] >= 0, f"NonNeg_y_{i+1}")
    for n in range(5):  # Adjusted to 0-based index
        m.addConstr(x_scenarios[i, n] >= 0, f"NonNeg_x_{i+1}_{n+1}")
        m.addConstr(y_scenarios[i, n] >= 0, f"NonNeg_y_{i+1}_{n+1}")



In [11]:
# Solve the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 116 rows, 36 columns and 166 nonzeros
Model fingerprint: 0x0aef7023
Variable types: 0 continuous, 36 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-01, 1e+00]
  Objective range  [4e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 116 rows and 36 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 1.15514e+06 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.155140000000e+06, best bound 1.155140000000e+06, gap 0.0000%


In [12]:
# Output the results
if m.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for v in m.getVars():
        print(f"{v.VarName}: {v.X}")

    print(f"Optimal Expected Revenue: {m.ObjVal}")
else:
    print("Optimal solution not found. Status code:", m.status)

Optimal solution found:
x[0]: 125.0
x[1]: 134.0
x[2]: 130.0
y[0]: 50.0
y[1]: 41.0
y[2]: 45.0
x_scenarios[0,0]: 79.0
x_scenarios[0,1]: 83.0
x_scenarios[0,2]: 87.0
x_scenarios[0,3]: 91.0
x_scenarios[0,4]: 95.0
x_scenarios[1,0]: 50.0
x_scenarios[1,1]: 53.0
x_scenarios[1,2]: 56.0
x_scenarios[1,3]: 59.0
x_scenarios[1,4]: 62.0
x_scenarios[2,0]: 54.0
x_scenarios[2,1]: 64.0
x_scenarios[2,2]: 64.0
x_scenarios[2,3]: 74.0
x_scenarios[2,4]: 84.0
y_scenarios[0,0]: 40.0
y_scenarios[0,1]: 42.0
y_scenarios[0,2]: 44.0
y_scenarios[0,3]: 46.0
y_scenarios[0,4]: 50.0
y_scenarios[1,0]: 30.0
y_scenarios[1,1]: 33.0
y_scenarios[1,2]: 36.0
y_scenarios[1,3]: 39.0
y_scenarios[1,4]: 41.0
y_scenarios[2,0]: 29.0
y_scenarios[2,1]: 33.0
y_scenarios[2,2]: 37.0
y_scenarios[2,3]: 41.0
y_scenarios[2,4]: 45.0
Optimal Expected Revenue: 1155140.0


In [13]:
# Output the results for VIP bookings for the Back-to-Back package
if m.status == GRB.OPTIMAL:
    vip_bookings_back_to_back = y[1].X
    print(f"Number of VIP bookings for the Back-to-Back package: {vip_bookings_back_to_back}")
else:
    print("Optimal solution not found. Status code:", m.status)

Number of VIP bookings for the Back-to-Back package: 41.0
